# Semi-Supervised Concept Drift

In [2]:
import pandas as pd
import numpy as np
import math

from typing import Tuple
import pandas as pd
import numpy as np 
from scipy import stats
import seaborn as sns
from tqdm import tqdm

from sklearn.metrics import confusion_matrix
from sklearn import svm

import warnings
warnings.filterwarnings("ignore")

# Read Data

In [3]:
### Subject 1

## Defining Filepaths
filepath = 'motion_sense'

### Creating a list of values for Subject 1 by randomly combining the different file

num_iter = 50
combined_df_sub_1 = pd.DataFrame()

curr_file_1 = filepath + '/sub_1_combined_data.csv'
curr_file_2 = filepath + '/sub_2_combined_data.csv'

curr_df_1 = pd.read_csv(curr_file_1)
curr_df_2 = pd.read_csv(curr_file_2)


curr_df_1.drop('Unnamed: 0.1',axis=1,inplace=True)
curr_df_2.drop('Unnamed: 0.1',axis=1,inplace=True)

curr_df_1.drop('Unnamed: 0',axis=1,inplace=True)
curr_df_2.drop('Unnamed: 0',axis=1,inplace=True)

curr_df_1.drop('outlier',axis=1,inplace=True)
curr_df_2.drop('outlier',axis=1,inplace=True)

curr_df_1.head()

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type
0,1.528132,-0.733896,0.696372,0.741895,0.669768,-0.031672,0.316738,0.778180,1.082764,0.294894,-0.184493,0.377542,dws
1,1.527992,-0.716987,0.677762,0.753099,0.657116,-0.032255,0.842032,0.424446,0.643574,0.219405,0.035846,0.114866,dws
2,1.527765,-0.706999,0.670951,0.759611,0.649555,-0.032707,-0.138143,-0.040741,0.343563,0.010714,0.134701,-0.167808,dws
3,1.516768,-0.704678,0.675735,0.760709,0.647788,-0.041140,-0.025005,-1.048717,0.035860,-0.008389,0.136788,0.094958,dws
4,1.493941,-0.703918,0.672994,0.760062,0.647210,-0.058530,0.114253,-0.912890,0.047341,0.199441,0.353996,-0.044299,dws


In [4]:
curr_df_1 = curr_df_1.rename(columns={"Unnamed: 0":"ind"})

In [5]:
curr_df_1["type"].value_counts()

sit    20584
std    12747
wlk    11112
jog     6428
ups     6336
dws     5105
Name: type, dtype: int64

In [6]:
curr_df_1.shape

(62312, 13)

In [7]:
curr_df_1.dtypes

attitude.roll         float64
attitude.pitch        float64
attitude.yaw          float64
gravity.x             float64
gravity.y             float64
gravity.z             float64
rotationRate.x        float64
rotationRate.y        float64
rotationRate.z        float64
userAcceleration.x    float64
userAcceleration.y    float64
userAcceleration.z    float64
type                   object
dtype: object

# Create Labels in Dataset

# sudden1

In [8]:
df_test=pd.read_csv('yifei_dataset_4.27_sudden30high200.csv')
df_test.head()

,Unnamed: 0,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type,type1_drift,type2_drift,type3_drift,type4_drift,type5_drift,type6_drift
0,0,-2.562144,-1.219983,-3.050600,-0.188176,0.939093,0.287564,-0.482461,-0.225021,-0.622184,0.184102,-0.178611,-0.341397,6,0,0,0,0,0,6
1,1,0.341979,-0.367058,0.116501,0.313013,0.358871,-0.879337,-0.015481,-0.017824,0.007785,0.002478,-0.002655,-0.013454,3,0,0,3,0,0,0
2,2,-2.104340,-1.225199,2.236723,-0.291675,0.940873,0.172289,0.320235,-0.378826,-0.375082,0.371951,0.409285,-0.202959,1,1,0,0,0,0,0
3,3,-2.749196,-0.528337,2.947486,-0.330262,0.504097,0.798006,4.186034,-6.205776,-2.193666,1.084580,2.214027,-2.280946,6,0,0,0,0,0,6
4,4,1.515196,-0.736980,0.985010,0.739357,0.672055,-0.041151,-2.032415,8.415040,-1.795768,-0.728463,-0.730648,1.059584,6,0,0,0,0,0,6


In [9]:
df_test["type"]

0        6
1        3
2        1
3        6
4        6
        ..
62307    4
62308    5
62309    1
62310    4
62311    4
Name: type, Length: 62312, dtype: int64

In [10]:
df_test1 = df_test.copy()
df_test1.drop(columns=["type","Unnamed: 0"], inplace=True)
test_X = df_test1.to_numpy()

In [11]:
df_test.type.value_counts()

3    19644
4    12518
5    11038
2     6726
6     5773
1     5616
0      997
Name: type, dtype: int64

# No Drift

In [12]:
df11=pd.read_csv('SampleDriftInjection/NoDrift.csv')
df11.head()

,Unnamed: 0,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type
0,3675,0.025033,-1.202555,1.409351,0.009010,0.932962,-0.359863,0.831373,-3.207910,1.773906,-0.087593,-0.088144,0.401702,1
1,47982,0.215725,-0.475396,-0.098294,0.190319,0.457691,-0.868503,-0.000831,0.009239,0.012150,0.001148,0.007473,-0.008587,3
2,32019,-1.882635,-1.172086,-2.556060,-0.369506,0.921562,0.119113,-0.963449,0.887903,0.567634,-0.449860,-0.484764,0.024732,5
3,4865,-1.488771,-1.300567,0.896711,-0.266055,0.963710,-0.021872,0.147023,0.327224,0.252847,0.155169,-0.258983,0.102073,2
4,25950,0.853153,-1.323178,0.511841,0.184644,0.969499,-0.161178,-0.083820,-0.338874,-0.219943,-0.064878,-0.024965,0.071929,4


In [13]:
df_test11 = df11.copy()
df_test11.drop(columns=["type","Unnamed: 0"], inplace=True)
test_X1 = df_test11.to_numpy()

# Original

In [14]:
# create labels
dictionary = {"sit":0, "std":1, "wlk":2, "jog":3, "ups":4, "dws":5}
dictionary

{'sit': 0, 'std': 1, 'wlk': 2, 'jog': 3, 'ups': 4, 'dws': 5}

In [15]:
curr_df_1["encoded"] = curr_df_1["type"].apply(lambda x: dictionary[x])

In [16]:
# shuffle the DataFrame rows
curr_df_11 = curr_df_1.sample(frac = 1, random_state=10)

In [17]:
curr_df_11[:600]["encoded"].value_counts()

0    202
1    119
2    105
4     61
5     57
3     56
Name: encoded, dtype: int64

In [18]:
curr_df_11.drop(columns=["type"], inplace=True)
curr_df_11.head()

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,encoded
34498,-3.119799,-0.435769,2.808867,-0.019756,0.422108,0.906330,-0.839569,-2.625599,-0.746496,0.357265,0.221188,-0.927754,2
17136,0.319598,-0.385717,0.047983,0.291102,0.376223,-0.879611,0.043646,-0.008660,-0.031626,0.002569,-0.000994,-0.016171,0
41849,-2.843994,-0.473762,3.103179,-0.260929,0.456237,0.850743,1.498496,0.761328,-0.171520,0.404773,0.104981,-1.186223,3
9595,0.356689,-0.442954,-0.102534,0.315475,0.428610,-0.846622,-0.025042,-0.034777,-0.094163,-0.001556,0.008524,-0.011319,0
35280,2.569419,-0.996903,2.479837,0.293962,0.839793,0.456435,1.360581,-1.830629,0.778301,-0.029421,1.071461,-1.307342,2


# Train base classifier

In [19]:
df_test.head()

,Unnamed: 0,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type,type1_drift,type2_drift,type3_drift,type4_drift,type5_drift,type6_drift
0,0,-2.562144,-1.219983,-3.050600,-0.188176,0.939093,0.287564,-0.482461,-0.225021,-0.622184,0.184102,-0.178611,-0.341397,6,0,0,0,0,0,6
1,1,0.341979,-0.367058,0.116501,0.313013,0.358871,-0.879337,-0.015481,-0.017824,0.007785,0.002478,-0.002655,-0.013454,3,0,0,3,0,0,0
2,2,-2.104340,-1.225199,2.236723,-0.291675,0.940873,0.172289,0.320235,-0.378826,-0.375082,0.371951,0.409285,-0.202959,1,1,0,0,0,0,0
3,3,-2.749196,-0.528337,2.947486,-0.330262,0.504097,0.798006,4.186034,-6.205776,-2.193666,1.084580,2.214027,-2.280946,6,0,0,0,0,0,6
4,4,1.515196,-0.736980,0.985010,0.739357,0.672055,-0.041151,-2.032415,8.415040,-1.795768,-0.728463,-0.730648,1.059584,6,0,0,0,0,0,6


In [20]:
df_test1.head()

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type1_drift,type2_drift,type3_drift,type4_drift,type5_drift,type6_drift
0,-2.562144,-1.219983,-3.050600,-0.188176,0.939093,0.287564,-0.482461,-0.225021,-0.622184,0.184102,-0.178611,-0.341397,0,0,0,0,0,6
1,0.341979,-0.367058,0.116501,0.313013,0.358871,-0.879337,-0.015481,-0.017824,0.007785,0.002478,-0.002655,-0.013454,0,0,3,0,0,0
2,-2.104340,-1.225199,2.236723,-0.291675,0.940873,0.172289,0.320235,-0.378826,-0.375082,0.371951,0.409285,-0.202959,1,0,0,0,0,0
3,-2.749196,-0.528337,2.947486,-0.330262,0.504097,0.798006,4.186034,-6.205776,-2.193666,1.084580,2.214027,-2.280946,0,0,0,0,0,6
4,1.515196,-0.736980,0.985010,0.739357,0.672055,-0.041151,-2.032415,8.415040,-1.795768,-0.728463,-0.730648,1.059584,0,0,0,0,0,6


In [21]:
def base_train(training, train_label, kernel_type="poly"):
    base_classifier = svm.SVC(kernel=kernel_type)
    base_classifier.probability=True
    base_classifier.fit(training, train_label)
    return base_classifier

In [22]:
# 1% training data for base classifier for training
# def base_train():
#     training = curr_df_11[:600]
#     train_label = training["encoded"]
#     training.drop(columns=["encoded"], inplace=True)
#     base_classifier = svm.SVC(kernel='linear')
#     base_classifier.probability=True
#     base_classifier.fit(training, train_label)
#     return base_classifier

# Manually Add Drift

In [23]:
# 99% of the rest of the data
others = curr_df_11[600:]
others.reset_index(inplace=True)
#label for the training set
other_label = others["encoded"]
others.drop(columns=["encoded"], inplace=True)
others.drop(columns=["index"], inplace=True)

In [24]:
others.shape

(61712, 12)

In [25]:
#add drift: by flipping sign of numerical features, 1000 chunks by 1000 chunks
drifts = range(0,61712,1000)
drifts = list(drifts)
len(drifts)

62

In [26]:
#index for unflipped rows
unflipped = []
for i in range(0,len(drifts),2):
    unflipped.extend(list(range(drifts[i],drifts[i+1])))
len(unflipped)

31000

In [27]:
#add drift: by flipping sign of numerical features, 1000 chunks by 1000 chunks
for index, row in others.iterrows():
    if index not in unflipped:
        others.iloc[index,:] = others.iloc[index,:].apply(lambda x: -x)

In [28]:
X = others.to_numpy()

# Apply Drift Detection Algorithm

## Fixed Reference Window

In [29]:
def FRW(X, n, base_classifier, retrain=False, num_std=1, train_labels=None):    
    #window size n
    sqrt_n = math.sqrt(n)

    #create sliding windows
    ref = []
    cur = []
    cur_idx = []
    idx = -1
    lst = []
    for i in X:
        idx += 1
        # calculate x
        prob = base_classifier.predict_proba([i])
        y_index = np.argmax(prob)
        y = [0]*6
        y[y_index] = 1
        distance = np.sum((np.array(y) - np.array(prob))**2).round(4)
        # move sliding window
        if len(ref) < n:
            ref.append(distance)
        elif len(cur) < n:
            cur.append(distance)
            cur_idx.append(idx)
        else:
            #move current window by 1
            cur.pop(0)
            cur.append(distance)
            cur_idx.pop(0)
            cur_idx.append(idx)
            #calculate window stats
            m_cur = np.mean(cur)
            m_ref = np.mean(ref)
            s_ref = np.std(ref)
            #determine if drift happens
            if m_cur >= m_ref + num_std*(s_ref/sqrt_n):
                lst.append(idx-n)
                #if to retrain the base classifier
                if retrain:
                    #learn a new base classifier
                    base_classifier.probability=True
                    base_classifier.fit(X[cur_idx], other_label[cur_idx])
                #initialize window as empty
                ref_idx = []
                ref = []
                cur = []
    return lst, len(lst)

## Moving Reference Window

In [30]:
def MRW(X, n, base_classifier, retrain=False, num_std=2, train_labels=None):    
    #window size n
    sqrt_n = math.sqrt(n)

    #create sliding windows
    ref = []
    cur = []
    cur_idx = []
    idx = -1
    lst = []
    for i in X:
        idx += 1
        # calculate x
        prob = base_classifier.predict_proba([i])
        y_index = np.argmax(prob)
        y = [0]*6
        y[y_index] = 1
        distance = np.sum((np.array(y) - np.array(prob))**2).round(4)
        # move sliding window
        if len(ref) < n:
            ref.append(distance)
        elif len(cur) < n:
            cur.append(distance)
            cur_idx.append(idx)
        else:
            #move reference window by 1
            ref.pop(0)
            ref.append(cur[0])
            #move current window by 1
            cur.pop(0)
            cur.append(distance)
            cur_idx.pop(0)
            cur_idx.append(idx)
            #append to current window 
            m_cur = np.mean(cur)
            m_ref = np.mean(ref)
            s_ref = np.std(ref)
            if m_cur >= m_ref + num_std*(s_ref/sqrt_n):
                lst.append(idx-n)
                #learn a new base classifier
                if retrain:
                    base_classifier.probability=True
                    base_classifier.fit(X[cur_idx], train_labels[cur_idx])
                #initialize window as empty
                cur_idx = []
                ref = []
                cur = [] 
    return lst, len(lst)

## Ensemble Reference Window

In [31]:
def ERW(X, n, base_classifier, retrain=False, num_std=4, train_labels=None):    
    #window size n
    sqrt_n = math.sqrt(n)
    c4 = math.sqrt(2/(n-1))*math.factorial(int(n/2) - 1)/math.factorial(int((n-1)/2) - 1)
    
    #create sliding windows
    ref = []
    cur = []
    cur_idx = []
    idx = -1
    lst = []
    m_lst = []
    s_lst = []

    for i in X:
        idx += 1
        # calculate x
        prob = base_classifier.predict_proba([i])
        y_index = np.argmax(prob)
        y = [0]*6
        y[y_index] = 1
        distance = np.sum((np.array(y) - np.array(prob))**2).round(4)
        # ensemble sliding window
        if len(ref) < n:
            ref.append(distance)
        elif len(cur) < n:
            cur.append(distance)
            cur_idx.append(idx)
        else:
            #move reference window by 1
            ref.pop(0)
            ref.append(cur[0])
            #move current window by 1
            cur.pop(0)
            cur.append(distance)
            cur_idx.pop(0)
            cur_idx.append(idx)
            #append to current window lists
            m_lst.append(np.mean(cur))
            s_lst.append(np.mean(cur)) 
            #calculate reference window mean
            m_ref = np.mean(ref)
            #determine if drift happens
            if np.mean(m_lst) + num_std*(np.std(m_lst)/sqrt_n)*c4 <= m_ref:
                lst.append(idx-n)
                if retrain:
                    #learn a new base classifier
                    base_classifier.probability=True
                    base_classifier.fit(X[cur_idx], train_labels[cur_idx])
                #initialize window as empty
                ref = []
                cur = []
                cur_idx = []
                m_lst = []
                s_lst = []   
    
    return lst, len(lst)

## Dynamically updating window size
- How to introduce metrics that determines the window size
    - if only 1 drifts detected in 2 window lengths: make the window size smaller
    - if half of the drifts are detected within one window, then increase the window size

In [32]:
def DMRW(X, base_classifier, retrain=False, num_std=2, train_labels=None):    
    #window size n
    min_window_size = 100
    max_window_size = 400
    cur_window_size = min_window_size
    ref_window_size = min_window_size
    cur_window_size = min_window_size

    #create sliding windows
    ref = []
    cur = []
    cur_idx = []
    idx = -1
    lst = []
    for i in X:
        idx += 1
        # calculate x
        prob = base_classifier.predict_proba([i])
        y_index = np.argmax(prob)
        y = [0]*6
        y[y_index] = 1
        distance = np.sum((np.array(y) - np.array(prob))**2).round(4)

        # move sliding windows
        if len(ref) < cur_window_size:
            ref.append(distance)
        elif len(cur) < cur_window_size:
            cur.append(distance)
            cur_idx.append(idx)

        #dynamically updating the window size
        elif len(lst)>2 and lst[-2]-lst[-1] > 2*cur_window_size:
            cur_window_size = max(min_window_size,cur_window_size-min_window_size)
        elif len(lst)>12 and lst[len(lst)//2]-lst[-1] < cur_window_size:
            cur_window_size = min(max_window_size,cur_window_size+min_window_size)

        else:
            #move reference window by 1
            ref.pop(0)
            ref.append(cur[0])
            #move current window by 1
            cur.pop(0)
            cur.append(distance)
            cur_idx.pop(0)
            cur_idx.append(idx)
            #append to current window 
            m_cur = np.mean(cur)
            m_ref = np.mean(ref)
            s_ref = np.std(ref)
            if m_cur >= m_ref + num_std*(s_ref/math.sqrt(cur_window_size)):
                lst.append(idx-cur_window_size)

                #learn a new base classifier
                if retrain:
                    base_classifier.probability=True
                    base_classifier.fit(X[cur_idx], train_labels[cur_idx])

                #initialize window as empty
                cur_idx = []
                ref = []
                cur = [] 
    return lst, len(lst)

In [38]:
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])
DMRW_lst_f, DMRW_length_f = DMRW(test_X, base_classifier,retrain=False, num_std=1)

In [39]:
DMRW_lst_f

[239, 456, 739, 940, 1198, 1640, 1841, 2159, 2360, 2647, 2905, 3247, 3485]

In [40]:
DMRW_length_f

13

In [45]:
MRW_lst_f,MRW_length_f = MRW(test_X, 100, base_classifier,False, 2)

print(MRW_lst_f)
print(MRW_length_f)

[239, 456, 739, 940, 1198, 1640, 1841, 2165, 2366, 2649, 2917, 3263, 3491, 3692, 4056, 4305, 4590, 4791, 5033, 5346, 5640, 5841, 6129, 6330, 6644, 6892, 7261, 7462, 7663, 8098, 8445, 8751, 8952, 9153, 9363, 9601, 9802, 10003, 10260, 10701, 10905, 11106, 11307, 11508, 11709, 11943, 12183, 12482, 12683, 12973, 13218, 13491, 13692, 13893, 14307, 14508, 14712, 15082, 15522, 15809, 16044, 16245, 16583, 16851, 17052, 17253, 17547, 17837, 18050, 18598, 18799, 19072, 19447, 19648, 19903, 20104, 20307, 20517, 20820, 21090, 21419, 21620, 21821, 22134, 22445, 22646, 22857, 23058, 23611, 24118, 24321, 24542, 24743, 24980, 25392, 25726, 26225, 26489, 26710, 26952, 27153, 27492, 27752, 28189, 28413, 28614, 28815, 29016, 29312, 29513, 29714, 29915, 30170, 30371, 30594, 30854, 31072, 31452, 31775, 32134, 32567, 32786, 33111, 33312, 33529, 33807, 34055, 34256, 34670, 34871, 35072, 35273, 35774, 36087, 36288, 36682, 36983, 37276, 37680, 38073, 38322, 38895, 39151, 39354, 39555, 39818, 40053, 40319, 4052

## Test Performance

In [95]:
#test combinations of window size
n=200
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

1
1.5
2
2.5
3
3.5
4


In [96]:
print(ERW_length_f_lst, FRW_length_f_lst, MRW_length_f_lst, DMRW_length_f_lst)

[120, 105, 87, 71, 48, 31, 25] [43, 26, 20, 25, 20, 15, 14] [123, 122, 121, 118, 116, 110, 109] [13, 13, 13, 13, 13, 13, 13]


In [97]:
for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s" % std_list[i])
    print("ERW: %s" % ERW_length_f_lst[i])
    print("FRW: %s" % FRW_length_f_lst[i])
    print("MRW: %s" % MRW_length_f_lst[i])
    print("DMRW: %s" % DMRW_length_f_lst[i])

Standard Deviation: 1
ERW: 120
FRW: 43
MRW: 123
DMRW: 13
Standard Deviation: 1.5
ERW: 105
FRW: 26
MRW: 122
DMRW: 13
Standard Deviation: 2
ERW: 87
FRW: 20
MRW: 121
DMRW: 13
Standard Deviation: 2.5
ERW: 71
FRW: 25
MRW: 118
DMRW: 13
Standard Deviation: 3
ERW: 48
FRW: 20
MRW: 116
DMRW: 13
Standard Deviation: 3.5
ERW: 31
FRW: 15
MRW: 110
DMRW: 13
Standard Deviation: 4
ERW: 25
FRW: 14
MRW: 109
DMRW: 13


In [37]:
def count_greater_than(numbers, threshold):
    count = 0
    for num in numbers:
        if num >= threshold:
            count += 1
    return count

In [38]:
def count_less_than(numbers, threshold):
    count = 0
    for num in numbers:
        if num < threshold:
            count += 1
    return count

In [99]:
for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

Standard Deviation: 1

ERW True Positive: 58
ERW False Positive: 62
FRW True Positive: 23
FRW False Positive: 20
MRW True Positive: 59
MRW False Positive: 64
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 51
ERW False Positive: 54
FRW True Positive: 21
FRW False Positive: 5
MRW True Positive: 59
MRW False Positive: 63
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 46
ERW False Positive: 41
FRW True Positive: 15
FRW False Positive: 5
MRW True Positive: 59
MRW False Positive: 62
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 34
ERW False Positive: 37
FRW True Positive: 22
FRW False Positive: 3
MRW True Positive: 57
MRW False Positive: 61
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 25
ERW False Positive: 23
FRW True Positive: 17
FRW False Positive: 3
MRW True Positive: 57
MRW False Positive: 59
DMRW True Positive

In [115]:
len(ERW_lst_t_lst)

8

In [116]:
#test combinations of window size
n=100
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

1
1.5
2
2.5
3
3.5
4
Standard Deviation: 1

ERW True Positive: 118
ERW False Positive: 132
FRW True Positive: 13
FRW False Positive: 20
MRW True Positive: 114
MRW False Positive: 127
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 108
ERW False Positive: 120
FRW True Positive: 11
FRW False Positive: 9
MRW True Positive: 114
MRW False Positive: 123
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 79
ERW False Positive: 89
FRW True Positive: 3
FRW False Positive: 6
MRW True Positive: 106
MRW False Positive: 117
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 63
ERW False Positive: 56
FRW True Positive: 2
FRW False Positive: 4
MRW True Positive: 101
MRW False Positive: 113
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 43
ERW False Positive: 43
FRW True Positive: 1
FRW False Positive: 4
MRW True Positive: 98
MRW False Po

In [117]:
#test combinations of window size
n=300
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

1
1.5
2
2.5
3
3.5
4
Standard Deviation: 1

ERW True Positive: 31
ERW False Positive: 43
FRW True Positive: 11
FRW False Positive: 6
MRW True Positive: 42
MRW False Positive: 42
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 30
ERW False Positive: 38
FRW True Positive: 26
FRW False Positive: 3
MRW True Positive: 42
MRW False Positive: 41
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 15
ERW False Positive: 29
FRW True Positive: 23
FRW False Positive: 3
MRW True Positive: 42
MRW False Positive: 39
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 10
ERW False Positive: 26
FRW True Positive: 19
FRW False Positive: 3
MRW True Positive: 40
MRW False Positive: 39
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 5
ERW False Positive: 16
FRW True Positive: 25
FRW False Positive: 3
MRW True Positive: 35
MRW False Positive: 35


### Testing with Gradual Drift

In [35]:
df_test_ay=pd.read_csv('yifei_dataset_4.27_gradual30high200.csv')
df_test_temp = df_test_ay.copy()
df_test_temp.drop(columns=["type","Unnamed: 0"], inplace=True)
test_X = df_test_temp.to_numpy()

In [39]:
#test combinations of window size
n=100
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

1
1.5
2
2.5
3
3.5
4
Standard Deviation: 1

ERW True Positive: 120
ERW False Positive: 122
FRW True Positive: 4
FRW False Positive: 22
MRW True Positive: 124
MRW False Positive: 127
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 112
ERW False Positive: 122
FRW True Positive: 1
FRW False Positive: 17
MRW True Positive: 117
MRW False Positive: 123
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 79
ERW False Positive: 103
FRW True Positive: 1
FRW False Positive: 8
MRW True Positive: 112
MRW False Positive: 119
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 67
ERW False Positive: 59
FRW True Positive: 1
FRW False Positive: 6
MRW True Positive: 106
MRW False Positive: 116
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 45
ERW False Positive: 47
FRW True Positive: 1
FRW False Positive: 6
MRW True Positive: 105
MRW False P

In [40]:
#test combinations of window size
n=200
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

1
1.5
2
2.5
3
3.5
4
Standard Deviation: 1

ERW True Positive: 59
ERW False Positive: 64
FRW True Positive: 4
FRW False Positive: 15
MRW True Positive: 57
MRW False Positive: 62
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 57
ERW False Positive: 58
FRW True Positive: 4
FRW False Positive: 12
MRW True Positive: 56
MRW False Positive: 61
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 37
ERW False Positive: 46
FRW True Positive: 5
FRW False Positive: 11
MRW True Positive: 56
MRW False Positive: 61
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 31
ERW False Positive: 41
FRW True Positive: 2
FRW False Positive: 14
MRW True Positive: 56
MRW False Positive: 59
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 18
ERW False Positive: 30
FRW True Positive: 2
FRW False Positive: 11
MRW True Positive: 55
MRW False Positive: 58

In [41]:
#test combinations of window size
n=300
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

1
1.5
2
2.5
3
3.5
4
Standard Deviation: 1

ERW True Positive: 31
ERW False Positive: 43
FRW True Positive: 20
FRW False Positive: 16
MRW True Positive: 38
MRW False Positive: 45
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 32
ERW False Positive: 41
FRW True Positive: 9
FRW False Positive: 7
MRW True Positive: 38
MRW False Positive: 43
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 15
ERW False Positive: 39
FRW True Positive: 18
FRW False Positive: 3
MRW True Positive: 36
MRW False Positive: 38
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 10
ERW False Positive: 32
FRW True Positive: 18
FRW False Positive: 3
MRW True Positive: 34
MRW False Positive: 38
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 6
ERW False Positive: 27
FRW True Positive: 13
FRW False Positive: 2
MRW True Positive: 34
MRW False Positive: 38


### Testing with Sudden (medium) Drift

In [42]:
df_test_ay=pd.read_csv('yifei_dataset_4.27_sudden30medium200.csv')
df_test_temp = df_test_ay.copy()
df_test_temp.drop(columns=["type","Unnamed: 0"], inplace=True)
test_X = df_test_temp.to_numpy()

In [43]:
#test combinations of window size
n=100
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

1
1.5
2
2.5
3
3.5
4
Standard Deviation: 1

ERW True Positive: 122
ERW False Positive: 125
FRW True Positive: 14
FRW False Positive: 23
MRW True Positive: 118
MRW False Positive: 128
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 112
ERW False Positive: 121
FRW True Positive: 4
FRW False Positive: 26
MRW True Positive: 115
MRW False Positive: 123
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 83
ERW False Positive: 103
FRW True Positive: 2
FRW False Positive: 15
MRW True Positive: 110
MRW False Positive: 122
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 70
ERW False Positive: 59
FRW True Positive: 2
FRW False Positive: 6
MRW True Positive: 105
MRW False Positive: 116
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 53
ERW False Positive: 47
FRW True Positive: 0
FRW False Positive: 6
MRW True Positive: 103
MRW False

In [44]:
#test combinations of window size
n=200
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

1
1.5
2
2.5
3
3.5
4
Standard Deviation: 1

ERW True Positive: 57
ERW False Positive: 64
FRW True Positive: 26
FRW False Positive: 10
MRW True Positive: 58
MRW False Positive: 64
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 55
ERW False Positive: 63
FRW True Positive: 24
FRW False Positive: 11
MRW True Positive: 58
MRW False Positive: 63
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 40
ERW False Positive: 46
FRW True Positive: 16
FRW False Positive: 11
MRW True Positive: 56
MRW False Positive: 61
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 31
ERW False Positive: 34
FRW True Positive: 10
FRW False Positive: 12
MRW True Positive: 56
MRW False Positive: 60
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 21
ERW False Positive: 24
FRW True Positive: 2
FRW False Positive: 5
MRW True Positive: 55
MRW False Positive:

In [45]:
#test combinations of window size
n=300
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

DMRW_lst_t_lst = []
DMRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

DMRW_length_t_lst = []
DMRW_length_f_lst = []

std_list = [1,1.5,2,2.5,3,3.5,4]

for num_std in std_list:
    print(num_std)
    
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    DMRW_lst_f,DMRW_length_f = DMRW(test_X, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #DMRW_lst_t_lst.append(MRW_lst_t)
    DMRW_lst_f_lst.append(DMRW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    
    #DMRW_length_t_lst.append(DMRW_length_t)
    DMRW_length_f_lst.append(DMRW_length_f)
    

for i in range(len(ERW_length_f_lst)):
    print("Standard Deviation: %s\n" % std_list[i])
    
    print("ERW True Positive: %s" % count_greater_than(ERW_lst_f_lst[i],len(test_X)/2))
    print("ERW False Positive: %s" % count_less_than(ERW_lst_f_lst[i],len(test_X)/2))
    
    print("FRW True Positive: %s" % count_greater_than(FRW_lst_f_lst[i],len(test_X)/2))
    print("FRW False Positive: %s" % count_less_than(FRW_lst_f_lst[i],len(test_X)/2))
    
    print("MRW True Positive: %s" % count_greater_than(MRW_lst_f_lst[i],len(test_X)/2))
    print("MRW False Positive: %s" % count_less_than(MRW_lst_f_lst[i],len(test_X)/2))
    
    print("DMRW True Positive: %s" % count_greater_than(DMRW_lst_f_lst[i],len(test_X)/2))
    print("DMRW False Positive: %s\n\n" % count_less_than(DMRW_lst_f_lst[i],len(test_X)/2))
    

1
1.5
2
2.5
3
3.5
4
Standard Deviation: 1

ERW True Positive: 31
ERW False Positive: 44
FRW True Positive: 8
FRW False Positive: 12
MRW True Positive: 39
MRW False Positive: 45
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 1.5

ERW True Positive: 30
ERW False Positive: 40
FRW True Positive: 16
FRW False Positive: 10
MRW True Positive: 39
MRW False Positive: 41
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2

ERW True Positive: 13
ERW False Positive: 40
FRW True Positive: 14
FRW False Positive: 6
MRW True Positive: 38
MRW False Positive: 40
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 2.5

ERW True Positive: 9
ERW False Positive: 32
FRW True Positive: 10
FRW False Positive: 5
MRW True Positive: 38
MRW False Positive: 38
DMRW True Positive: 0
DMRW False Positive: 13


Standard Deviation: 3

ERW True Positive: 4
ERW False Positive: 23
FRW True Positive: 12
FRW False Positive: 5
MRW True Positive: 36
MRW False Positive: 38


### Comments
- Larger Standard Deviations, less false positives but also less drifts.
- Bayesian optimization: randomly sample window size/ other combinations instead of discrete ones.
- Maybe try other algorithms and also experiment with window sizes.
- Base before calibrating, user defined parameters, parameters could be adjusted with confidence(optional)

**Other Requirements:**
- Precision and recalls besides accuracy
- types of drifts
- PEP8, docstrings and typeints, along with pseudo code documentations and test files


## Comments on Algorithm results
- **On the Motion Sense Dataset, if we define drift as greater than 1 standard deviation of the reference window**:
    - Using Fixed Reference window, then we could detect 6/61 drifts in the concept drift, that is about 10% of our manually created drifts.
    - Using Moving Reference Window, then we could detect 46/61 drifts in the concept drift, that is about 75% of our manually created drifts.
    - Using Ensemble Reference Window, then we could detect 30/61 drifts in the concept drift, that is about 50% of our manually created drifts.
    
- **Next Steps**:
    - Experiment with Window size
    - Experiment types of concept drifts

- **Further Directions**:
    - Adaptive thresholds for numbers of std and window sizes
    - maybe 1%/0.1% of data is the optimal window size
